In [1]:
from strategies import simulator
from strategies import random_nodes

config = simulator.SimulationConfig(
        n_samples=2,
        n_batches=5,
        max_interventions=2,
        n_nodes=10,
        edge_prob=.1,
        n_dags=100
    )
simulator.simulate(random_nodes.random_strategy, config, 1)


StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.